In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [5]:
df=pd.read_csv("/content/household_power_consumption (3).csv")


In [6]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16-12-2006,17:24:00,4.216,0.418,234.84,18.4,0,1,17.0
1,16-12-2006,17:25:00,5.36,0.436,233.63,23,0,1,16.0
2,16-12-2006,17:26:00,5.374,0.498,233.29,23,0,2,17.0
3,16-12-2006,17:27:00,5.388,0.502,233.74,23,0,1,17.0
4,16-12-2006,17:28:00,3.666,0.528,235.68,15.8,0,1,17.0


In [7]:
df.describe()

,Sub_metering_3
count,64497.000000
mean,7.466161
std,8.656777
min,0.000000
25%,0.000000
50%,0.000000
75%,17.000000
max,20.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64505 entries, 0 to 64504
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   64505 non-null  object 
 1   Time                   64505 non-null  object 
 2   Global_active_power    64504 non-null  object 
 3   Global_reactive_power  64504 non-null  object 
 4   Voltage                64504 non-null  object 
 5   Global_intensity       64504 non-null  object 
 6   Sub_metering_1         64503 non-null  object 
 7   Sub_metering_2         64503 non-null  object 
 8   Sub_metering_3         64497 non-null  float64
dtypes: float64(1), object(8)
memory usage: 4.4+ MB


In [9]:
df.isna().sum()

Date                     0
Time                     0
Global_active_power      1
Global_reactive_power    1
Voltage                  1
Global_intensity         1
Sub_metering_1           2
Sub_metering_2           2
Sub_metering_3           8
dtype: int64

In [10]:
df.dropna(inplace=True)

In [11]:
df['Global_active_power']=df['Global_active_power'].apply(lambda x:float(x))
df['Global_reactive_power']=df['Global_reactive_power'].apply(lambda x:float(x))
df['Voltage']=df['Voltage'].apply(lambda x:float(x))

In [12]:
df_clear=df.dropna()
df_clear

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16-12-2006,17:24:00,4.216,0.418,234.84,18.4,0,1,17.0
1,16-12-2006,17:25:00,5.360,0.436,233.63,23,0,1,16.0
2,16-12-2006,17:26:00,5.374,0.498,233.29,23,0,2,17.0
3,16-12-2006,17:27:00,5.388,0.502,233.74,23,0,1,17.0
4,16-12-2006,17:28:00,3.666,0.528,235.68,15.8,0,1,17.0
...,...,...,...,...,...,...,...,...,...
64499,15-01-2007,12:30:00,1.506,0.226,240.70,6.2,0,0,18.0
64500,15-01-2007,12:31:00,1.504,0.226,240.65,6.2,0,0,17.0
64501,15-01-2007,12:32:00,1.510,0.226,240.73,6.2,0,0,18.0
64502,15-01-2007,12:33:00,1.512,0.230,241.23,6.2,0,0,18.0


In [13]:
x=df.iloc[:,3:8]
y=df.iloc[:,2]

In [14]:
x

,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2
0,0.418,234.84,18.4,0,1
1,0.436,233.63,23,0,1
2,0.498,233.29,23,0,2
3,0.502,233.74,23,0,1
4,0.528,235.68,15.8,0,1
...,...,...,...,...,...
64499,0.226,240.70,6.2,0,0
64500,0.226,240.65,6.2,0,0
64501,0.226,240.73,6.2,0,0
64502,0.230,241.23,6.2,0,0


In [15]:
y

0        4.216
1        5.360
2        5.374
3        5.388
4        3.666
         ...  
64499    1.506
64500    1.504
64501    1.510
64502    1.512
64503    1.510
Name: Global_active_power, Length: 64497, dtype: float64

In [16]:
# Splitting data into train and test set
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=0)

In [17]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
xtrain=sc.fit_transform(xtrain)
xtest=sc.fit_transform(xtest)

In [18]:
# Make the ANN
import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [19]:
# Initialize 
ann=Sequential()


In [21]:
ann=Sequential()
ann.add(Dense(units=10,activation='relu'))
ann.add(Dense(units=1))
ann.compile(optimizer='adam',loss='mean_squared_error')
ann.fit(xtrain,ytrain,epochs=20,batch_size=38)
ypred=ann.predict(xtest)

Epoch 1/20
1358/1358 [==============================] - 5s 2ms/step - loss: 1.2128
Epoch 2/20
1358/1358 [==============================] - 2s 2ms/step - loss: 0.0210
Epoch 3/20
1358/1358 [==============================] - 2s 2ms/step - loss: 0.0097
Epoch 4/20
1358/1358 [==============================] - 2s 1ms/step - loss: 0.0060
Epoch 5/20
1358/1358 [==============================] - 2s 2ms/step - loss: 0.0041
Epoch 6/20
1358/1358 [==============================] - 2s 2ms/step - loss: 0.0033
Epoch 7/20
1358/1358 [==============================] - 2s 2ms/step - loss: 0.0031
Epoch 8/20
1358/1358 [==============================] - 2s 2ms/step - loss: 0.0030
Epoch 9/20
1358/1358 [==============================] - 2s 2ms/step - loss: 0.0029
Epoch 10/20
1358/1358 [==============================] - 2s 2ms/step - loss: 0.0029
Epoch 11/20
1358/1358 [==============================] - 2s 2ms/step - loss: 0.0029
Epoch 12/20
1358/1358 [==============================] - 2s 2ms/step - loss: 0.0028
E